In [1]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])


gs


CommaSeparatedListOutputParser
- 콤마 기준으로 나눔 + csv파일로 저장할때 용이

In [5]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import csv

# 콤마로 구분된 리스트 출력 파서 초기화
output_parser = CommaSeparatedListOutputParser()

# 출력 형식 지침 가져오기
format_instructions = output_parser.get_format_instructions()

# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

# OpenAI 모델 설정
model = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# 프롬프트, 모델, 출력 파서를 연결하여 체인 생성
chain = prompt | model | output_parser

# "AI 관련 기술"에 대한 체인 호출 실행
result = chain.invoke({"subject": "AI 관련 기술"})

# 쉼표로 구분된 리스트 출력
print(" AI 관련 기술 목록:")
print(result)

# 결과 활용 예시: CSV 파일로 저장
csv_filename = "./data/ai_technologies.csv"
with open(csv_filename, "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["AI 기술"])  # 헤더 추가
    for item in result:
        writer.writerow([item])

print(f" '{csv_filename}' 파일로 저장 완료!")


AuthenticationError: Error code: 401 - {'error': {'message': 'Invalid API Key', 'type': 'invalid_request_error', 'code': 'invalid_api_key'}}

JSON output parser
- JSON 형식으로 구조화된 데이터 생성 및 API응답이나 데이터분석 용이

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
import json

# JSON 출력 파서 초기화
parser = JsonOutputParser()

# 프롬프트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 과학 분야 전문가 AI입니다. 질문에 대해 체계적이고 간결한 답변을 JSON 형식으로 제공하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

# JSON 출력 형식 지침을 프롬프트에 적용
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# OpenAI 모델 설정
model = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
# 프롬프트, 모델, 출력 파서를 연결하는 체인 생성
chain = prompt | model | parser

# 질문 설정 (우주 탐사 관련 질문)
question = "최근 10년간 진행된 주요 우주 탐사 미션 3가지를 알려주세요. \
각 미션의 이름은 `mission_name`에, 목표는 `goal`에, 주관 기관은 `agency`에 담아 주세요."

# 체인 실행 및 JSON 응답 받기
response = chain.invoke({"question": question})

# JSON 데이터 출력
print(json.dumps(response, indent=4, ensure_ascii=False))


AuthenticationError: Error code: 401 - {'error': {'message': 'Invalid API Key', 'type': 'invalid_request_error', 'code': 'invalid_api_key'}}

PandasDataFrameOutputParser
- pandas DataFrame과 LLM을 연동하여 데이터를 쉽게 조회하고 분석할 수 있도록 돕는 출력 파서
- .csv 데이터프레임의 데이터를 분석하기 가능

In [6]:
import pprint
from typing import Any, Dict
import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 웹에서 Titanic 데이터셋 불러오기
df = pd.read_csv('data/titanic.csv')

format_instructions = parser.get_format_instructions()

# Pandas DataFrame Output Parser 설정
parser = PandasDataFrameOutputParser(dataframe=df)
df.head(2)

NameError: name 'parser' is not defined

In [8]:

# ChatOpenAI 모델 초기화
model = ChatOpenAI(temperature=0, model_name="gpt-4o")

# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template="""
    You are a helpful assistant that interacts whti a Pandas DataFrame.
    The DataFrame contains the following columns: {columns}.
    You task is to answer the user's query by generating a command in the following format:
    {format_instructions}\n{query}\n",
    
    User Query: {query}
    """,
    input_variables=["query"],
    partial_variables={
        "format_instructions": format_instructions,
        "columns": ", ".join(df.columns)
    },
)


# 체인 생성
chain = prompt | model | parser

try:
    print("Name 컬럼 출력")
    df_query = "Show the Name Column"

    parser_output = chain.invoke({"query": df_query})
    print(type(parser_output))
    print(parser_output)

    print("첫번째 행 출력")
    df_query2 = "Show first row"

    parser_output2 = chain.invoke({"query": df_query2})
    print(parser_output2)
except Exception as e:
    print(f"오류발생: {e}")

NameError: name 'parser' is not defined

PydanticOutputParser
- 결과 구조를 미리 지정해서 사용 가능
- poetry add pydantic (%pip install pydantic) 으로 다운로드 후 사용

In [2]:
# 필수 라이브러리 설치
#%pip install pydantic

from dotenv import load_dotenv
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser

from pydantic import BaseModel, Field
from typing import List

# 출력 구조를 정의하는 Pydantic 모델
class MovieRecommendation(BaseModel):
    movie_title: str = Field(description="추천 영화 제목")
    reason: str = Field(description="추천 이유")
    genre: List[str] = Field(description="영화 장르")
    estimated_rating: float = Field(description="10점 만점에서 예상 평점")
    
# Pydantic 출력 파서 초기화
parser = PydanticOutputParser(pydantic_object=MovieRecommendation)

# 프롬프트 템플릿 설정
template = """
다음 사용자 요청에 따라 영화를 추천해주세요.
요청: {query}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)

# 파서의 지시사항을 프롬프트에 주입
prompt = prompt.partial(
    format_instructions=parser.get_format_instructions()
)

# ChatOpenAI 모델 초기화
#model = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.7
)

# 체인 구성 및 실행
query = "1990년대 클래식한 느낌의 공포 영화 추천해줘"
chain = prompt | model | parser
output = chain.invoke({"query": query})

# 결과 출력
print(f"추천 영화: {output.movie_title}")
print(f"추천 이유: {output.reason}")
print(f"장르: {', '.join(output.genre)}")
print(f"예상 평점: {output.estimated_rating}/10")
    

추천 영화: The Silence of the Lambs
추천 이유: 1990년대 클래식한 느낌의 공포 영화로, 오스카 최우수 작품상을 수상한 명작입니다.
장르: 공포, 스릴러, 미스터리
예상 평점: 9.5/10


StructuredOutputParser
- 결과를 미리 구조화해서 이런식으로 출력받고싶다. 를 설정해놓고 사용하는 것

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

from pprint import pprint

# 출력 구조 정의 (평점, 장점, 단점, 요약)
response_schemas = [
    ResponseSchema(name="rating", description="5점 만점에서 예상 평점"),
    ResponseSchema(name="pros", description="리뷰에서 언급된 장점 3가지를 리스트로 출력"),
    ResponseSchema(name="cons", description="리뷰에서 언급된 단점 3가지를 리스트로 출력"),
    ResponseSchema(name="summary", description="리뷰를 한 문장으로 요약")
]

# 파서 초기화
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()

print("출력 형식 지시사항:")
print(format_instructions)


출력 형식 지시사항:
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"rating": string  // 5점 만점에서 예상 평점
	"pros": string  // 리뷰에서 언급된 장점 3가지를 리스트로 출력
	"cons": string  // 리뷰에서 언급된 단점 3가지를 리스트로 출력
	"summary": string  // 리뷰를 한 문장으로 요약
}
```


In [5]:

# 프롬프트 템플릿
template = """
다음 제품 리뷰를 분석하세요. 리뷰 내용: {review}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)

# 모델 초기화 (temperature=0.5로 설정해 일관성 있는 출력)
#model = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.7
)

# 테스트 리뷰 데이터
review = """
이 스마트폰은 배터리 수명이 정말 좋아서 하루 종일 사용해도 충전이 필요 없었어요. 
카메라 화질도 선명하고, 특히 야간 모드가 훌륭합니다. 
다만 가격이 조금 비싸고, 무게가 200g이 넘어서 손이 피곤할 수 있어요.
"""

# 체인 실행
chain = prompt | model | parser

output = chain.invoke({"review": review})

# 결과 출력 (Pretty Print)
print("===== 분석 결과 =====")
pprint(output)

===== 분석 결과 =====
{'cons': ['가격이 비싸다', '무게가 무거워서 손이 피곤할 수 있다', '단점 3번째 항목 없음'],
 'pros': ['배터리 수명이 좋다', '카메라 화질이 선명하다', '야간 모드가 훌륭하다'],
 'rating': '4',
 'summary': '스마트폰의 배터리 수명과 카메라 화질이 좋지만, 가격이 비싸고 무겁다는 리뷰입니다.'}


DatetimeOutputParser
> 날짜나 시간 정보 파싱
- pip install python-dateutil
- poetry add python-dateutil

In [9]:
#%pip install python-dateutil

from dotenv import load_dotenv
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import DatetimeOutputParser
from datetime import datetime, timedelta

print(datetime.today())

# 출력 파서 초기화 (시간대 포함 가능)
datetime_parser = DatetimeOutputParser()
format_instructions = datetime_parser.get_format_instructions()

print("날짜 출력 형식 지시사항:")
print(format_instructions)

# 프롬프트 템플릿
# 현재 날짜를 명시적으로 프롬프트에 주입
current_date = datetime.now().strftime("%Y-%m-%d")
template = f"""
현재 날짜: {current_date}
다음 텍스트에서 날짜/시간 정보를 추출하세요. 상대적 표현(예: '다음 주 금요일')은 현재 날짜를 기준으로 계산합니다.
텍스트: {{text}}

{{format_instructions}}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)
print(prompt)

2025-06-19 15:52:54.178061
날짜 출력 형식 지시사항:
Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 1029-12-27T11:29:40.596313Z, 0450-12-10T19:59:15.365004Z, 1661-12-16T11:17:36.244019Z

Return ONLY this string, no other words!
input_variables=['text'] input_types={} partial_variables={'format_instructions': "Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 1029-12-27T11:29:40.596313Z, 0450-12-10T19:59:15.365004Z, 1661-12-16T11:17:36.244019Z\n\nReturn ONLY this string, no other words!"} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'text'], input_types={}, partial_variables={}, template="\n현재 날짜: 2025-06-19\n다음 텍스트에서 날짜/시간 정보를 추출하세요. 상대적 표현(예: '다음 주 금요일')은 현재 날짜를 기준으로 계산합니다.\n텍스트: {text}\n\n{format_instructions}\n"), additional_kwargs={})]


In [11]:

# 모델 초기화 (temperature=0.1로 설정해 정확한 날짜 출력 강조)
#model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.1
)

# 테스트 데이터 (다양한 날짜 형식 포함)
texts = [
    "회의는 2025년 6월 15일 오후 2시에 예정되어 있습니다.",
    "프로젝트 마감일은 다음 주 금요일입니다.",
    "행사 시작: 7/25/2025 18:00 KST",
    "3일 후에 시스템 점검이 진행됩니다."
]

# 체인 실행 및 결과 출력
chain = prompt | model | datetime_parser

for text in texts:
    print(f"\n원본 텍스트: {text}")
    output = chain.invoke({"text": text})
    print(f"추출된 날짜: {output.strftime('%Y-%m-%d %H:%M:%S %Z')}")

    



원본 텍스트: 회의는 2025년 6월 15일 오후 2시에 예정되어 있습니다.
추출된 날짜: 2025-06-15 14:00:00 

원본 텍스트: 프로젝트 마감일은 다음 주 금요일입니다.
추출된 날짜: 2025-06-27 00:00:00 

원본 텍스트: 행사 시작: 7/25/2025 18:00 KST
추출된 날짜: 2025-07-25 18:00:00 

원본 텍스트: 3일 후에 시스템 점검이 진행됩니다.
추출된 날짜: 2025-06-22 00:00:00 


In [13]:
# 이벤트 추출용 프롬프트
current_date = datetime.now().strftime("%Y-%m-%d")

event_template = """
현재 날짜: {current_date}
다음 텍스트에서 모든 이벤트의 날짜/시간을 추출하세요. 각 이벤트는 이름과 날짜를 포함해야 합니다.
텍스트: {text}

출력 형식:
- 이벤트명: [이름]
- 날짜: [YYYY-MM-DD HH:MM:SS]
"""

event_prompt = ChatPromptTemplate.from_template(event_template)
event_chain = event_prompt | model

# 예시 텍스트 (여러 이벤트 포함)
event_text = """
12월 10일에 크리스마스 마켓이 열리고, 12월 24일에는 크리스마스 이브 파티가 있습니다.
또한 내년 1월 1일 00:00에 새해 카운트다운이 진행될 예정입니다.
"""

print(event_chain.invoke({"current_date":current_date, "text": event_text}).content)

현재 날짜는 2025-06-19이므로, 제공한 텍스트의 날짜를 현 년도로 변경해줘야 합니다.

- 이벤트명: 크리스마스 마켓
- 날짜: 2025-12-10 00:00:00

- 이벤트명: 크리스마스 이브 파티
- 날짜: 2025-12-24 00:00:00

- 이벤트명: 새해 카운트다운
- 날짜: 2026-01-01 00:00:00


EnumOutputParser
- 미리 정의된 선택지 중 해당하는거로 분류해주는 것

In [17]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import EnumOutputParser, OutputFixingParser
from langchain.schema import OutputParserException

from enum import Enum
from pprint import pprint

# 감정 클래스 정의 (Enum)
class Sentiment(str, Enum):
    POSITIVE = "긍정"
    NEGATIVE = "부정"
    NEUTRAL = "중립"

# EnumOutputParser 초기화
enumParser = EnumOutputParser(enum=Sentiment)
format_instructions = enumParser.get_format_instructions()

print("감정 분류 출력 형식:")
print(format_instructions)

# 프롬프트 템플릿
template = """
당신은 텍스트 감정 분석 전문가입니다.
다음 텍스트의 감정을 분석하고, 반드시 아래 세 가지 중 하나의 단어로만 답변하세요.

텍스트: "{text}"

{format_instructions}

중요 규칙:
1. 반드시 "긍정", "부정", "중립" 중 하나의 단어만 출력하세요
2. 다른 설명이나 부가 설명을 추가하지 마세요
3. 이모지나 특수문자도 포함하지 마세요
4. 오직 하나의 단어만 출력하세요

답변:"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)   
print(prompt)

감정 분류 출력 형식:
Select one of the following options: 긍정, 부정, 중립
input_variables=['text'] input_types={} partial_variables={'format_instructions': 'Select one of the following options: 긍정, 부정, 중립'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'text'], input_types={}, partial_variables={}, template='\n당신은 텍스트 감정 분석 전문가입니다.\n다음 텍스트의 감정을 분석하고, 반드시 아래 세 가지 중 하나의 단어로만 답변하세요.\n\n텍스트: "{text}"\n\n{format_instructions}\n\n중요 규칙:\n1. 반드시 "긍정", "부정", "중립" 중 하나의 단어만 출력하세요\n2. 다른 설명이나 부가 설명을 추가하지 마세요\n3. 이모지나 특수문자도 포함하지 마세요\n4. 오직 하나의 단어만 출력하세요\n\n답변:'), additional_kwargs={})]


In [18]:
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0  # 일관성을 위해 0으로 설정
)

# OutputFixingParser로 안정성 향상
fixing_parser = OutputFixingParser.from_llm(parser=enumParser, llm=model)

print("모델 및 파서 설정 완료")

# 테스트 텍스트
texts = [
    "이 제품 정말 좋아요! 완전 만족스러워요.",
    "서비스가 너무 느리고 불친절했습니다.",
    "오늘은 비가 온다네요.",
    "배송은 빠르지만 품질이 아쉽습니다.",
    "최고의 경험이었습니다!",
    "완전 실망했어요... 최악이에요"
]

print(f"테스트할 텍스트 {len(texts)}개 준비 완료")

모델 및 파서 설정 완료
테스트할 텍스트 6개 준비 완료


In [19]:
# 안전한 감정 분석 함수 (에러 처리 포함)
def safe_sentiment_analysis(text, use_fixing_parser=True):
    """안전한 감정 분석 함수 - 에러 처리 포함"""
    try:
        # 기본 체인 생성
        chain = prompt | model | (fixing_parser if use_fixing_parser else enumParser)
        
        # 분석 실행
        result = chain.invoke({"text": text})
        return result, None
        
    except OutputParserException as e:
        return None, f"파싱 오류: {str(e)[:100]}..."
    except Exception as e:
        return None, f"일반 오류: {str(e)[:100]}..."

# 실제 감정 분석 실행 (API 키 필요)
def run_sentiment_analysis():
    """실제 감정 분석 실행"""
    print("=== 실제 감정 분석 결과 ===")
    
    success_count = 0
    total_count = len(texts)
    
    for i, text in enumerate(texts, 1):
        print(f"\n{i}. 텍스트: {text}")
        
        # OutputFixingParser 사용
        result, error = safe_sentiment_analysis(text, use_fixing_parser=True)
        
        if result:
            print(f"   감정: {result.value} ")
            success_count += 1
        else:
            print(f"   오류: {error} ")
            
            # 기본 파서로 재시도
            print("   기본 파서로 재시도...")
            result2, error2 = safe_sentiment_analysis(text, use_fixing_parser=False)
            
            if result2:
                print(f"   감정: {result2.value} (기본 파서 성공)")
                success_count += 1
            else:
                print(f"   재시도 실패: {error2} ")
    
    print(f"\n=== 결과 요약 ===")
    print(f"성공: {success_count}/{total_count} ({success_count/total_count*100:.1f}%)")
    print(f"실패: {total_count-success_count}/{total_count}")

# 실제 분석 실행 (API 키가 있는 경우)
try:
    run_sentiment_analysis()
except Exception as e:
    print("API 키가 설정되지 않았거나 네트워크 오류:")
    print("실제 실행을 위해서는 OpenAI API 키를 설정하세요.")
    print(f"오류 상세: {e}")

=== 실제 감정 분석 결과 ===

1. 텍스트: 이 제품 정말 좋아요! 완전 만족스러워요.
   감정: 긍정 

2. 텍스트: 서비스가 너무 느리고 불친절했습니다.
   감정: 부정 

3. 텍스트: 오늘은 비가 온다네요.
   감정: 중립 

4. 텍스트: 배송은 빠르지만 품질이 아쉽습니다.
   감정: 부정 

5. 텍스트: 최고의 경험이었습니다!
   감정: 긍정 

6. 텍스트: 완전 실망했어요... 최악이에요
   감정: 부정 

=== 결과 요약 ===
성공: 6/6 (100.0%)
실패: 0/6


BooleanOutputParser
- Yes/No로 판별

In [24]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import BooleanOutputParser
from langchain_core.output_parsers import StrOutputParser

# Boolean 파서 초기화
boolParser = BooleanOutputParser()

# 수동으로 포맷 지시사항 정의 (LangChain 버전 이슈 회피)
format_instructions = """
출력은 반드시 다음 중 하나여야 합니다:
- `True`: 모든 조건 충족 시
- `False`: 하나라도 조건 불충족 시

예시:
True  # 모든 조건 만족
False # 조건 불만족
"""

# 승인/거부 결정 프롬프트 템플릿
template = """
다음 대출 신청자를 평가하세요. 조건을 모두 충족하면 `True`, 아니면 `False`를 출력하세요.

### 조건:
1. 나이 >= {min_age}세
2. 신용 점수 >= {min_credit_score}
3. 월 수입 >= ${min_income}

### 신청자 정보:
{applicant_details}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)
print(prompt)

input_variables=['applicant_details', 'min_age', 'min_credit_score', 'min_income'] input_types={} partial_variables={'format_instructions': '\n출력은 반드시 다음 중 하나여야 합니다:\n- `True`: 모든 조건 충족 시\n- `False`: 하나라도 조건 불충족 시\n\n예시:\nTrue  # 모든 조건 만족\nFalse # 조건 불만족\n'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['applicant_details', 'format_instructions', 'min_age', 'min_credit_score', 'min_income'], input_types={}, partial_variables={}, template='\n다음 대출 신청자를 평가하세요. 조건을 모두 충족하면 `True`, 아니면 `False`를 출력하세요.\n\n### 조건:\n1. 나이 >= {min_age}세\n2. 신용 점수 >= {min_credit_score}\n3. 월 수입 >= ${min_income}\n\n### 신청자 정보:\n{applicant_details}\n\n{format_instructions}\n'), additional_kwargs={})]


In [25]:

model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.1
)

chain = prompt | model | boolParser

# 테스트 케이스
test_cases = [
    {
        "min_age": 18,
        "min_credit_score": 700,
        "min_income": 3000,
        "applicant_details": """
        - 이름: 김철수
        - 나이: 25세
        - 신용 점수: 750
        - 월 수입: $3,500
        """
    },
    {
        "min_age": 18,
        "min_credit_score": 700,
        "min_income": 3000,
        "applicant_details": """
        - 이름: 이영희
        - 나이: 17세
        - 신용 점수: 680
        - 월 수입: $2,800
        """
    }
]

# 거부 사유 생성 프롬프트 (출력 형식 명시적으로 지정)
reason_template = """
다음 대출 신청 거부 사유를 1문장으로 설명하세요. 반드시 다음 형식으로 답변해야 합니다:

[거부 사유]: [사유 내용]

### 신청자 정보:
{applicant_details}

### 조건:
- 최소 나이: {min_age}세
- 최소 신용 점수: {min_credit_score}
- 최소 월 수입: ${min_income}
"""
reason_prompt = ChatPromptTemplate.from_template(reason_template)
reason_chain = reason_prompt | model | StrOutputParser()

# 체인 확장 (Boolean 파서와 분리)
def get_decision_with_reason(input_dict):
    # 1. 먼저 Boolean 결정
    decision = chain.invoke(input_dict)
    
    # 2. 거부 시에만 사유 생성
    if not decision:
        try:
            reason = reason_chain.invoke({
                "applicant_details": input_dict["applicant_details"],
                "min_age": input_dict["min_age"],
                "min_credit_score": input_dict["min_credit_score"],
                "min_income": input_dict["min_income"]
            })
            return decision, reason
        except Exception as e:
            return decision, f"거부 사유 생성 실패: {str(e)}"
    return decision, "모든 조건을 충족했습니다."

# 테스트 (안전한 실행)
try:
    decision, reason = get_decision_with_reason(test_cases[0])
    print(f"\n결과: {'승인' if decision else '거부'}")
    print(f"사유: {reason}")
except Exception as e:
    print(f"오류 발생: {str(e)}")

오류 발생: BooleanOutputParser expected output value to include either YES or NO. Received ### 대출 신청자 평가

#### 조건:
1. 나이 >= 18세
2. 신용 점수 >= 700
3. 월 수입 >= $3000

#### 신청자 정보:
- 이름: 김철수
- 나이: 25세
- 신용 점수: 750
- 월 수입: $3,500

### 평가 코드

```python
def evaluate_loan_application(name, age, credit_score, monthly_income):
    # 조건 1: 나이 >= 18세
    condition1 = age >= 18
    
    # 조건 2: 신용 점수 >= 700
    condition2 = credit_score >= 700
    
    # 조건 3: 월 수입 >= $3000
    condition3 = monthly_income >= 3000
    
    # 모든 조건 충족 시 True 반환
    return condition1 and condition2 and condition3

# 신청자 정보
name = "김철수"
age = 25
credit_score = 750
monthly_income = 3500

# 평가 결과
result = evaluate_loan_application(name, age, credit_score, monthly_income)

print(result)  # True
```

### 결과 설명

- 나이: 25세 (조건 만족)
- 신용 점수: 750 (조건 만족)
- 월 수입: $3,500 (조건 만족)

모든 조건을 충족하므로 결과는 `True`입니다..
